In [1]:
# import libraries
import numpy as np
np.set_printoptions(precision = 4, suppress = True)
import pandas as pd
import os
from datetime import datetime
%matplotlib qt

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# import modules
from widgets import CandlestickWidget

In [2]:
out_data_dir = "../data/out"
in_data_dir = "../data/in"

In [3]:
def get_data_points(csv_path, sep=",", date_col="time"):    
    # read file
    data_points = pd.read_csv(csv_path, sep)
    
    # convert time
    data_points[date_col] = data_points[date_col].apply(lambda x: datetime.utcfromtimestamp(x))
    
    # set index
    data_points.set_index(date_col, inplace=True)

    return data_points

In [4]:
open_points = get_data_points(os.path.join(out_data_dir, "open_points.csv"))
close_points = get_data_points(os.path.join(out_data_dir, "closed_points.csv"))
mean_price_points = get_data_points(os.path.join(out_data_dir, "mean_price_points.csv"))
indic_points = get_data_points(os.path.join(out_data_dir, "indicator_values.csv"))

In [5]:
candlestick_points = get_data_points(os.path.join(in_data_dir, "btc-usdt-30-min.csv"), sep=";")

In [6]:
# reindex
open_points = open_points.reindex(candlestick_points.index)
closed_points = close_points.reindex(candlestick_points.index)
mean_price_points = mean_price_points.reindex(candlestick_points.index)
indic_points = indic_points.reindex(candlestick_points.index)

In [7]:
pair = "BTC/USDT"
interval = "30 min"
title = f"{pair}\n{interval}"

In [8]:
candles = CandlestickWidget(candlestick_points, indic_points, mean_price_points, open_points, closed_points, title)

In [9]:
candles.show()